# Some basic Theano brain teasers
There is a n-dim tensor variable, you `dimshuffle` with pattern `x`, then you take its sum at some axis `y`, what do you get? Does it sound like a good puzzle to you? Then let's figure it out.

But first we should `import` some magics...

In [1]:
import theano
import numpy as np

from theano import tensor as T

### Custom tensor type 
It's very easy to define a new tensor type in Theano via the `broadcastable` argument.

In [2]:
dvector  = T.TensorType(broadcastable=(False,), dtype='float32')
drow     = T.TensorType(broadcastable=(True, False), dtype='float32')
dcol     = T.TensorType(broadcastable=(False, True), dtype='float32')
dtensor3 = T.TensorType(broadcastable=(False, False, False), dtype='float32')

### Reductions
If you do `foo(axis=n)` then the dimension `n` is collapsed and deleted with all values in the new matrix equal to the sum of the corresponding collapsed values.

In [3]:
# new variable of type dtensor3
x = dtensor3()

# sum of x along axis 0
s0 = x.sum(axis=0)

# feed x some value 
x_val = np.reshape(np.arange(3 * 3 * 3, dtype='float32'),(3, 3, 3))
dic   = {x : x_val}

# s0 is the sum of 3 sub 3x3 matrix
print "x:\n{0}".format(x_val)
print "s0:\n{0}".format(s0.eval(dic))

x:
[[[  0.   1.   2.]
  [  3.   4.   5.]
  [  6.   7.   8.]]

 [[  9.  10.  11.]
  [ 12.  13.  14.]
  [ 15.  16.  17.]]

 [[ 18.  19.  20.]
  [ 21.  22.  23.]
  [ 24.  25.  26.]]]
s0:
[[ 27.  30.  33.]
 [ 36.  39.  42.]
 [ 45.  48.  51.]]


In [ ]:
# teaser 1
s1 = x.sum(axis=1)
s2 = x.sum(axis=0).sum(axis=0)
s3 = x.sum(axis=(0,1))
s4 = x.sum(axis=0).sum(axis=1)

# some common reductions
total = x.sum()
marginals = x.sum(axis=(0,2))
mx = x.max(axis=1)

### Dimshuffle
Now let's moving on to `dimshuffle`. Basically, `dimshuffle(d1, d2, d3)` takes the elements in `d1` and scatter them into `d2`, etc.

In [4]:
# warm up
y0 = x.dimshuffle((1, 0, 2))

# a little harder
y1 = x.dimshuffle((2, 0, 1))

print "y0:\n{0}".format(y0.eval(dic))
print "y1:\n{0}".format(y1.eval(dic))

y0:
[[[  0.   1.   2.]
  [  9.  10.  11.]
  [ 18.  19.  20.]]

 [[  3.   4.   5.]
  [ 12.  13.  14.]
  [ 21.  22.  23.]]

 [[  6.   7.   8.]
  [ 15.  16.  17.]
  [ 24.  25.  26.]]]
y1:
[[[  0.   3.   6.]
  [  9.  12.  15.]
  [ 18.  21.  24.]]

 [[  1.   4.   7.]
  [ 10.  13.  16.]
  [ 19.  22.  25.]]

 [[  2.   5.   8.]
  [ 11.  14.  17.]
  [ 20.  23.  26.]]]


In [ ]:
# teaser 2
y2 = x.dimshuffle((0, 2, 1))
y3 = x.dimshuffle((1, 2, 0))
y4 = x.dimshuffle((2, 1, 0))

# adding to larger tensor
a = T.matrix()
b = a.dimshuffle(0, 1, 'x') + x

a_val = np.ones((3,3), dtype='float32')
print "b:\n{0}".format(b.eval({x : x_val, a : a_val}))

### Shared variable
A shared variable is the buffer that stores a numerical value for a Theano variable

In [5]:
state = theano.shared(0.)

inc = T.scalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])
print "state0: {0}".format(state.get_value())

accumulator(1)
print "state1: {0}".format(state.get_value())

state.set_value(100.)
print "state2: {0}".format(state.get_value())

fn_of_state = state * 2 + inc
foo = T.scalar(dtype=state.dtype)
skip_shared = theano.function([inc, foo], fn_of_state, givens=[(state, foo)]) # try replace givens with updates

state0: 0.0
state1: 1.0
state2: 100.0


In [ ]:
# teaser 3
skip_shared(1, 2)
print "state3: {0}".format(state.get_value())